# Class 7 - Metaprogramming

Today we'll cover metaprogramming in Julia.  This should answer some questions about what's really going on when you use a macro such as `@simd` or a expression such as `:hello`.  

# What is Metaprogramming?

Briefly, [metaprogramming](https://en.wikipedia.org/wiki/Metaprogramming) refers to writing programs to write programs.  This is poweful because it allows you to write less code for repetitive tasks, and even write domain-specific languages.  [Template Metaprogramming](https://en.wikipedia.org/wiki/Template_metaprogramming) is one example of this.  e.g.

In [1]:
function cme257fn(a::T, b::T) where T
    a + T(2)*b
end
;

In [4]:
cme257fn(1.,1.)

3.0

allows you to write a function for any number of types `T`.  We've already seen quite a bit of this, and today we'll talk more about macros and expressions.

## Reference

Today we'll mostly cover material that can be found in the [metaprogramming](https://docs.julialang.org/en/stable/manual/metaprogramming/) section of Julia's documentation.  Many examples will be drawn from the documentation.

# Expressions - How Text Becomes a Program

Everything you put into a Jupyter notebook, a `.jl` file, or the Julia REPL starts as a string.

In [5]:
prog = "1 + 1"
typeof(prog)

String

Julia then parses this string to determine what to do with the input

In [6]:
exp = Meta.parse(prog)

:(1 + 1)

In [7]:
typeof(exp)

Expr

In [8]:
eval(exp)

2

You can think of expressions as "quoting" code, not evaluating it.  For example `1+1` evalueates to 2, but the expression `:(1+1)` is akin to the satement `'1+1' is an equation`, which doesn't refer to the result of the evaluation, but the statement itself

In [9]:
# expressions have a symbol that denotes the type of expression
@show typeof(exp.head)
exp.head

typeof(exp.head) = Symbol


:call

In [10]:
# expression arguments may be symbols, other expressions, or literal values
exp.args

3-element Array{Any,1}:
  :+
 1  
 1  

This means that the program itself is stored in a data structure accessible through Julia!

You can always run your program using `eval`

In [11]:
@show exp
eval(exp)

exp = :(1 + 1)


2

Some useful tools for working with expressions are `dump` and `Meta.show_sexpr`.  These are both ways of visualizing the [abstract syntax tree (AST)](https://en.wikibooks.org/wiki/Introducing_Julia/Metaprogramming#The_Abstract_Syntax_Tree) that is generated when code is parsed.

In [12]:
# more detailed printout of expression
dump(exp)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


In [13]:
exp = Meta.parse("(1+1)/2")

:((1 + 1) / 2)

In [14]:
# s-expressions represent expression as tree
Meta.show_sexpr(exp)

(:call, :/, (:call, :+, 1, 1), 2)

In [15]:
dump(exp)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol /
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 1
    3: Int64 2


# Symbols, Quoting

Symbols are denoted by the colon `:` symbol.  These are [interned strings](https://en.wikipedia.org/wiki/String_interning) which is used to build expressions.  

In [16]:
@show typeof(:hello)
@show typeof(Symbol("hello"))
@show sym = Symbol("hello", "_world", 10)
@show typeof(sym)
;

typeof(:hello) = Symbol
typeof(Symbol("hello")) = Symbol
sym = Symbol("hello", "_world", 10) = :hello_world10
typeof(sym) = Symbol


A second role for the `:` character is for "quoting", which helps build expressions


In [17]:
exp = :(1 + 2 * 3)
dump(exp)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 2
        3: Int64 3


quoing can also be done using the `quote` keyword

In [19]:
exp = quote
    x = 1
    x + x
end
@show typeof(exp)
@show exp
#eval(exp)

typeof(exp) = Expr
exp = quote
    #= In[19]:2 =#
    x = 1
    #= In[19]:3 =#
    x + x
end


quote
    #= In[19]:2 =#
    x = 1
    #= In[19]:3 =#
    x + x
end

In [20]:
dump(exp)

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[19]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol x
        2: Int64 1
    3: LineNumberNode
      line: Int64 3
      file: Symbol In[19]
    4: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Symbol x
        3: Symbol x


# Interpolation

Just like you can interpolate in expressions by using `$`, just like you can in strings

In [21]:
x = 1
"x = $x"

"x = 1"

In [22]:
a = 1
exp = :($a + b)

:(1 + b)

In [24]:
b = 2
eval(exp)

3

In [25]:
# you can also interpolate expressions.
exp1 = :(1+1)
exp2 = :($exp1/2)

:((1 + 1) / 2)

# Exercise 1

1. try putting a simple for-loop in a quote.  What does it look like? Try using `dump` and `Meta.show_sexpr`
2. Create an expression for `1 + (2+3)` starting with the expression `:(2+3)`
3. use quoting with `:` to examine the expression for the function `sum257{T}(a::T,b::T) = a+b`.  Try rewriting the function using the `function` keyword, and creating an expression using a `quote` block.

In [37]:
 c = 1
for_exp = quote
    for i = 1:2
        global c+= i
    end
    c
end
#@show Meta.show_sexpr(for_exp)
#dump(for_exp);

In [38]:
eval(for_exp)

4

In [39]:
exp1 = :(2+3 )
@show exp2 = Expr(:call, :+, 1, exp1)
eval(exp2)

exp2 = Expr(:call, :+, 1, exp1) = :(1 + (2 + 3))


6

In [40]:
@show exp2_alt = :(1+$exp1)
eval(exp2_alt)

exp2_alt = $(Expr(:quote, :(1 + $(Expr(:$, :exp1))))) = :(1 + (2 + 3))


6

In [41]:
:(sum257(a::T,b::T) where T = a+b)

:((sum257(a::T, b::T) where T) = begin
          #= In[41]:1 =#
          a + b
      end)

In [42]:
expr = quote
    function sum257(a::T,b::T) where T
        return a+b
    end
end
@show(expr);

expr = quote
    #= In[42]:2 =#
    function sum257(a::T, b::T) where T
        #= In[42]:3 =#
        return a + b
    end
end


# Functions of Expressions

Since expressions are a data type in Julia, you can create functions of expresssions

In [43]:
# expression for binary operation
function bin_exp(op, arg1, arg2)
   return Expr(:call, op, arg1, arg2) 
end

exp = bin_exp(:+, 1,2)
@show exp
@show eval(exp)

exp = bin_exp(:*, 2, :(1+1))
exp2 = bin_exp(:*, 2, bin_exp(:+,1,1))
@show exp
@show exp == exp2
@show eval(exp)
;

exp = :(1 + 2)
eval(exp) = 3
exp = :(2 * (1 + 1))
exp == exp2 = true
eval(exp) = 4


In [44]:
:(1 + 1)
bin_exp(:+, 1, 1)

:(1 + 1)

# Macros

Macros give you a way to generate code to be included in a program.  They are evaluated directly, and do not require a call to `eval()`.  They are created using the `macro` keyword.

In [45]:
macro helloworld()
    return :( println("Hello World!") )
end

@helloworld (macro with 1 method)

In [46]:
@helloworld

Hello World!


In [47]:
# macros can take arguments
macro hello(name)
    return :(println("Hello ", $name))
end

@hello (macro with 1 method)

In [48]:
@hello "CME 257"

Hello CME 257


In [49]:
# you can also call a macro with function-like syntax
@hello("CME 257")

Hello CME 257


There are tools to help you build, understand, and debug macros - the function `macroexpand()` and the macro `@macroexpand`

In [52]:
macroexpand(Main, :(@hello("CME 257")))

:(Main.println("Hello ", "CME 257"))

In [53]:
# the macro version is easier to call
@macroexpand @hello "CME 257"

:(Main.println("Hello ", "CME 257"))

Macros are executed at parse time, not at run time (or compile time).  This means they don't add overhead to a function call (since they generate actual code).  However, they may add overhead to using a module, or including a file.  

In [55]:
macro twostep(arg)
   println("I execute at parse time. The argument is: ", arg)
   return :(println("I execute at runtime. The argument is: ", $arg))
end



@twostep (macro with 1 method)

In [56]:
ex = @macroexpand @twostep 1+1

I execute at parse time. The argument is: 1 + 1


:(Main.println("I execute at runtime. The argument is: ", 1 + 1))

In [57]:
eval(ex)

I execute at runtime. The argument is: 2


## Examples

We've already seen the `@assert` macro

In [58]:
# does nothing
@assert 1==1
# throws error
@assert 1==2
;

AssertionError: AssertionError: 1 == 2

The [following example](https://docs.julialang.org/en/v1/manual/metaprogramming/#Building-an-advanced-macro-1) is given as a way to implement the `@assert` macro

In [59]:
macro assert2(ex)
    return :( $ex ? nothing : throw(AssertionError($(string(ex)))) ) #this is an inline if-else statement
end
# does nothing
@assert2 1==1
# does something
@assert2 1==2
;

AssertionError: AssertionError: 1 == 2

In [60]:
@macroexpand @assert 1==2

:(if 1 == 2
      nothing
  else
      Base.throw(Base.AssertionError("1 == 2"))
  end)

At runtime, the expression `ex` is spliced into the return statement in two places (`$ex`, `string(ex)`). The expression is evaluated, and depending on the result the macro either does nothing or throws an error.

if you do something more complicated in a macro, you may wish to use local variables to avoid creating new variables outside the macro.  Note you can use the `local` keyword in more general contexts.

In [61]:
@time 1+1

  0.000006 seconds (4 allocations: 160 bytes)


2

In [65]:
@macroexpand @time 1+1

quote
    #= util.jl:154 =#
    local #36#stats = Base.gc_num()
    #= util.jl:155 =#
    local #38#elapsedtime = Base.time_ns()
    #= util.jl:156 =#
    local #37#val = 1 + 1
    #= util.jl:157 =#
    #38#elapsedtime = Base.time_ns() - #38#elapsedtime
    #= util.jl:158 =#
    local #39#diff = Base.GC_Diff(Base.gc_num(), #36#stats)
    #= util.jl:159 =#
    Base.time_print(#38#elapsedtime, (#39#diff).allocd, (#39#diff).total_time, Base.gc_alloc_count(#39#diff))
    #= util.jl:161 =#
    Base.println()
    #= util.jl:162 =#
    #37#val
end

In [1]:
macro time2(ex)
    return quote
        local t0 = time()
        local val = $ex
        local t1 = time()
        println("elapsed time: ", t1-t0, " seconds")
        val
    end
end

@time2 (macro with 1 method)

In [2]:
a = 1
b = 2
@time2 a + b

elapsed time: 9.059906005859375e-6 seconds


3

In [67]:
t0 = 1
t1 = 2
#uh-oh!
@macroexpand @time2 t0 + t0

quote
    #= In[62]:3 =#
    local #43#t0 = Main.time()
    #= In[62]:4 =#
    local #44#val = #43#t0 + #43#t0
    #= In[62]:5 =#
    local #45#t1 = Main.time()
    #= In[62]:6 =#
    Main.println("elapsed time: ", #45#t1 - #43#t0, " seconds")
    #= In[62]:7 =#
    #44#val
end

We see that even though we tried to avoid a name conflict, we still ran into trouble because we interpolated the expression in the macro definition.  We can avoid this by escaping the expression with `esc()`

In [68]:
macro time2(ex)
    return quote
        local t0 = time()
        local val = $(esc(ex))
        local t1 = time()
        println("elapsed time: ", t1-t0, " seconds")
        val
    end
end

@time2 (macro with 1 method)

In [71]:
t0 = 1
t1 = 2
@time2 t0+t0

elapsed time: 3.0994415283203125e-6 seconds


2

# Exercise 2

1. Create a macro `@show2` that does the same thing as the `@show` macro
2. compare expressions generated in `@assert` and `@assert2` with the same input. You'll likely want to use `@macroexpand` and `dump()`
3. compare the expressions generated by `@show` and your `@show2` macro with the same input.

In [15]:
macro show2(ex)
    local name = string(ex)
    local val =  eval(ex)
    println(string(name, " = ", val))   
    val
end

@show2 (macro with 1 method)

In [16]:
@show2 1+1

1 + 1 = 2


2

In [17]:
@show 1+1

1 + 1 = 2


2

# Code Generation

Metaprogramming can save you quite a bit of time and effort when it comes to writing repetitive code

In [86]:
import Base: +, -, *, /
struct cme257type
    val::Float64
end

for op in (:+, :-, :*, :/)
   eval(quote
        ($op)(a::cme257type, b::cme257type) = cme257type($(op)(a.val, b.val))
    end)
end

In [87]:
+(a::cme257type, b::cme257type) = cme257type(+(a.val, b.val))

+ (generic function with 162 methods)

In [88]:
a = cme257type(1)
b = cme257type(2)
@show a * b
@show a+b

a * b = cme257type(2.0)
a + b = cme257type(3.0)


cme257type(3.0)

there's a handy `@eval` macro that can be used to do the same thing

In [89]:
# option 1
for op in (:+, :-, :*, :/)
   @eval begin
        ($op)(a::cme257type, b::cme257type) = cme257type($(op)(a.val, b.val))
    end
end

# option 2
for op in (:+, :-, :*, :/)
   @eval ($op)(a::cme257type, b::cme257type) = cme257type($(op)(a.val, b.val))
end

In [90]:
a = cme257type(1.0)
b = cme257type(2.0)
a / b

cme257type(0.5)

# Generated Functions

Generated functions are a powerful way of creating functions, based on the types of the arguments.  Note that you have to be careful to avoid unintended side effects with them, so refer to the [documentation](https://docs.julialang.org/en/v1/manual/metaprogramming/#Generated-functions-1) if you indend to use them seriously, beyond this brief introduction.


In a sentence, a generated function acts the same as a normal function, with the exception that multiple dispatch happens when the function is called. In other words, Julia only creates a new method when the function is called with differently-typed objects. This can be useful when optimizing code, as Julia won't create a new method until the need arises.

In [91]:
@generated function foo(x)
   Core.println(x)
   return :(x * x)
end

foo (generic function with 1 method)

In [93]:
x = foo(2);

note that the generated function is generally only produced once. If you run this cell twice with the same type of input, you probably won't see the type output again.  However, there's no guarantee that this will happen, which is why the functions can't have any side effects 

In [94]:
x

4

For the next example, note that you can create a function with variable length arguments using `...`

In [95]:
function test_fn(I::Int64...)
    println("n inputs = ", length(I))
    c = 0
    for i = 1:length(I)
       println("  $i: $(I[i])") 
       c += I[i]
    end
    return c
end

test_fn (generic function with 1 method)

In [98]:
test_fn(1, 5, 3,4)

n inputs = 4
  1: 1
  2: 5
  3: 3
  4: 4


13

Now, for the example, which is described in [the metaprogramming documentaion](https://docs.julialang.org/en/v1/manual/metaprogramming).  Recall that in Julia, arrays are stored in column-major format.  We can change the indexing of an array to linear indexing with `LinearIndices()`.

In [99]:
using LinearAlgebra
A = [1 2;3 4]
# two ways of accessing an array: single index and multi-index
@show A[2]
@show A[2,1]
# the following gives us the conversion from the multi-index to the single index
b = LinearIndices(A)
@show b[3];
@show b[1,2];

A[2] = 3
A[2, 1] = 3
b[3] = 3
b[1, 2] = 3


In [109]:
a = tuple(1,2)
Iterators.

```
product(iters...)
```

Return an iterator over the product of several iterators. Each generated element is a tuple whose `i`th element comes from the `i`th argument iterator. The first iterator changes the fastest.

# Examples

```jldoctest
julia> collect(Iterators.product(1:2, 3:5))
2×3 Array{Tuple{Int64,Int64},2}:
 (1, 3)  (1, 4)  (1, 5)
 (2, 3)  (2, 4)  (2, 5)
```


For a 2-dimensional array `A` with dimensions `(m,n)`, the formula for indices `A[i,j]` is `i + m*(j-1)`.  We'll see how this generalizes shortly.

Now, the clever use of a generated function: we can write a single generated function that will produce the output of `sub2ind` that dispatches based on the number of dimensions.

In [100]:
@generated function lin_ind(dims::NTuple{N}, I::Integer...) where N
   length(I) == N || return :(error("partial indexing is unsupported"))
   ex = :(I[$N] - 1)
   for i = (N - 1):-1:1
       ex = :(I[$i] - 1 + dims[$i] * $ex)
   end
   return :($ex + 1)
end

lin_ind (generic function with 1 method)

In [101]:
lin_ind((2,2), 1, 2)

3

Note that we could do something similar with a non-generated function, but then we would evaluate the loop at run-time, instead of complie time.  Since there is some overhead associated with loops, for frequently called functions this can give performance benefits.

In [46]:
function sub2ind_loop(dims::NTuple{N}, I::Integer...) where N
    length(I) == N || return error("partial indexing is unsupported")
    ind = I[N] - 1
    for i = N-1:-1:1
        ind = I[i]-1 + dims[i]*ind
    end
    return ind + 1
end


sub2ind_loop (generic function with 1 method)

# Exercise 3

Recall how we were able to call library functions e.g.  `ccall((:cos, "libm"), Float64, (Float64,), 1.0)`. Generate a set of wrappers for the following functions in libm - `cos, sin, acos, asin, tan, fabs, cosh` (just work with `Float64` inputs).  To avoid conflicts with existing funcitons, name the wrappers `cme257op` where `op` is the operation name.  Add some more functions with a single `Float64` input from [here](https://en.wikipedia.org/wiki/C_mathematical_functions)

Hint: you can quote an expression `ex` using `Expr(:quote, ex)` - this is useful for the first argument of `ccall`

If you have time, check out how TensorFlow.jl [wraps the TensorFlow library](https://github.com/malmaud/TensorFlow.jl/blob/master/src/ops/math.jl) using metaprogramming

# Further Reading

* [Wikibook on Metaprogramming in Julia](https://en.wikibooks.org/wiki/Introducing_Julia/Metaprogramming)
* [On Machine Learning and Programming Languages](https://julialang.org/blog/2017/12/ml&pl)
* Check out how TensorFlow.jl [wraps the TensorFlow library](https://github.com/malmaud/TensorFlow.jl/blob/master/src/ops/math.jl)

In [47]:
# Exercise 3 example solution
ops = (:cos, :sin, :acos, :asin, :tan, :fabs, :cosh)
for op in ops
    fname = Symbol(:cme257,op)
    println("generating $fname")
    opn = Expr(:quote, op)
    @eval begin
        $(fname)(x::Float64) = ccall(($opn, "libm"), Float64, (Float64,), x)
    end
end

generating cme257cos
generating cme257sin
generating cme257acos
generating cme257asin
generating cme257tan
generating cme257fabs
generating cme257cosh
